In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

seed = 0

In [ ]:
import os
for root, dirs, files in os.walk("/kaggle/input", topdown=True):
   for name in dirs:
      print(os.path.join(root, name))

In [ ]:
path = '/kaggle/input/allstate-purchase-prediction-challenge/'

df = pd.read_csv(path+'train.csv.zip')
#df = pd.read_csv(path+'est_v2.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
missing_cols = [col for col in df.columns if df[col].isnull().any() == True]
df[missing_cols].describe()

In [ ]:
df[missing_cols].apply(lambda x: x.value_counts())

In [ ]:
df['car_value'].describe()

In [ ]:
df['car_value'].value_counts()

In [ ]:
df['car_value'] = df['car_value'].fillna('e')

In [ ]:
df[df.risk_factor == df.C_previous].shape[0]

should we use a model to predict risk factor? That would possible introduce some bias / colinearity into the data but allows us to make accurate guesses

In [ ]:
df.corr()['risk_factor']

In [ ]:
df[['age_oldest', 'age_youngest']].describe()

the negative correlation between risk_factor and the ages means the higher the age the lower the risk

given the risk_factor has fairly uniform distribution across all 4 values, we will fill missing values with the ... If age_oldest is in the top quartile (between 75% and 100%) they will have a risk_factor of 1.

In [ ]:
age = df['age_oldest']
quantiles = age.quantile([0,.25,.50,.75,1])
quantiles

In [ ]:
def age_to_risk_factor(age):
    for i, quantile in enumerate(quantiles):
        if age < quantile:
            return i
    return 4

In [ ]:
risk_null = df[df.risk_factor.isnull()]

new_quantiles = risk_null['age_oldest'].apply(age_to_risk_factor)
risk_null['risk_factor'] = new_quantiles
# for index in risk_null.index:
#         #risk_null['risk_factor'] = risk_null['risk_factor'].apply(lambda x: age_to_risk_factor(risk_null['age_oldest']))
#         risk_null.iloc[index]['risk_factor'] = age_to_risk_factor(risk_null.iloc[index]['age_oldest'])

In [ ]:
new_quantiles.head()

In [ ]:
# risk_null['risk_factor'] = age_to_risk_factor(risk_null['age_oldest'])
df.iloc[risk_null.index]['risk_factor'] = new_quantiles
#df.risk_factor.isnull().sum()

In [ ]:
df.iloc[risk_null.index, -15:]

In [ ]:
df.iloc[15]['risk_factor'] = 3

In [ ]:
df.isnull().sum()

In [ ]:
df['C_previous'] = df['C_previous'].fillna(3.0)

In [ ]:
dp_mean = df['duration_previous'].mean()
df['duration_previous'] = df['duration_previous'].fillna(dp_mean)

In [ ]:
df.risk_factor = df.risk_factor.fillna(np.random.randint(1,5))
df.risk_factor.value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby(['customer_ID']).sum()['record_type'].value_counts()

each customer makes 1 and only 1 purchase

In [ ]:
car_value_dummies = pd.get_dummies(df['car_value'], prefix='car_value')
df = pd.concat([df.drop('car_value', axis=1), car_value_dummies], axis=1)

df.shape

In [ ]:
df['time'] = df['time'].apply(lambda x: x[:x.find(':')]).astype('int')
df['time'].head()

In [ ]:
df = df.rename(columns={'time':'hour'})

In [ ]:
df['hour'].value_counts().sort_index()